# Día 13: Pipelines

[Pipelines](https://www.kaggle.com/alexisbcook/pipelines?utm_medium=email&utm_source=gamma&utm_campaign=thirty-days-of-ml&utm_content=day-13)

## Objetivo de la práctica
aprender a usar `pipelines` para limpiar su código de modelado.

## Introducción

Las canalizaciones (Pipelines) son una forma sencilla de mantener organizado el código de modelado y preprocesamiento de datos.

Específicamente, una canalización agrupa los pasos de preprocesamiento y modelado para que pueda usar todo el paquete como si fuera un solo paso.

Muchos científicos de datos piratean modelos sin canalizaciones, pero las canalizaciones tienen algunos beneficios importantes. Aquellos incluyen:

1. **Código más limpio**: La contabilidad de los datos en cada paso del preprocesamiento puede resultar complicada.   
  * Con una canalización, no necesitará realizar un seguimiento manual de sus datos de capacitación y validación en cada paso.

2. **Menos errores**: Hay menos oportunidades de aplicar incorrectamente un paso u olvidar un paso de procesamiento previo.

3. **Más fácil de producir:** Puede ser sorprendentemente difícil hacer la transición de un modelo de un prototipo a algo que se pueda implementar a escala.   
  * No entraremos en las muchas preocupaciones relacionadas aquí, pero las canalizaciones pueden ayudar.

4. **Más opciones para la validación del modelo:** Verá un ejemplo en el siguiente tutorial, que cubre la validación cruzada.

## Ejemplo

Como en el tutorial anterior, trabajaremos con el conjunto de datos de Melbourne Housing.

No nos centraremos en el paso de carga de datos.

En su lugar, puede imaginar que está en un punto en el que ya tiene los datos de entrenamiento y validación en 
* `X_train`,   
* `X_valid`,   
* `y_train` e   
* `y_valid`.

In [1]:
import pandas as pd 
data = pd.read_csv('melb_data.csv')
# https://www.kaggle.com/dansbecker/melbourne-housing-snapshot

In [2]:
data.head(3) 

,Suburb,Address,Rooms,Type,Price,Method,SellerG,Date,Distance,Postcode,...,Bathroom,Car,Landsize,BuildingArea,YearBuilt,CouncilArea,Lattitude,Longtitude,Regionname,Propertycount
0,Abbotsford,85 Turner St,2,h,1480000.0,S,Biggin,3/12/2016,2.5,3067.0,...,1.0,1.0,202.0,NaN,NaN,Yarra,-37.7996,144.9984,Northern Metropolitan,4019.0
1,Abbotsford,25 Bloomburg St,2,h,1035000.0,S,Biggin,4/02/2016,2.5,3067.0,...,1.0,0.0,156.0,79.0,1900.0,Yarra,-37.8079,144.9934,Northern Metropolitan,4019.0
2,Abbotsford,5 Charles St,3,h,1465000.0,SP,Biggin,4/03/2017,2.5,3067.0,...,2.0,0.0,134.0,150.0,1900.0,Yarra,-37.8093,144.9944,Northern Metropolitan,4019.0


In [3]:
from sklearn.model_selection import train_test_split

## Objetivo separado de los predictores

In [5]:
y = data.Price
X = data.drop(['Price'], axis = 1)

## Divide los datos en subconjuntos de entrenamiento y validación

In [6]:
X_train_full, X_valid_full, y_train, y_valid = train_test_split(X, y, train_size=0.8, 
                                                                test_size=0.2,
                                                                random_state=0)

## Seleccionar columnas categóricas 

"Cardinalidad" significa el número de valores únicos en una columna.

Seleccione columnas categóricas con cardinalidad relativamente baja (conveniente pero arbitraria)

In [7]:
categorical_cols = [cname for cname in X_train_full.columns 
                    if X_train_full[cname].nunique() < 10 and 
                        X_train_full[cname].dtype == "object"]

## Seleccionar columnas numéricas

In [8]:
numerical_cols = [cname for cname in X_train_full.columns 
                  if X_train_full[cname].dtype in ['int64', 
                                                   'float64']]

# Mantener solo las columnas seleccionadas

In [9]:
my_cols = categorical_cols + numerical_cols
X_train = X_train_full[my_cols].copy()
X_valid = X_valid_full[my_cols].copy()

In [10]:
X_train.head(3) 

,Type,Method,Regionname,Rooms,Distance,Postcode,Bedroom2,Bathroom,Car,Landsize,BuildingArea,YearBuilt,Lattitude,Longtitude,Propertycount
12167,u,S,Southern Metropolitan,1,5.0,3182.0,1.0,1.0,1.0,0.0,NaN,1940.0,-37.85984,144.9867,13240.0
6524,h,SA,Western Metropolitan,2,8.0,3016.0,2.0,2.0,1.0,193.0,NaN,NaN,-37.85800,144.9005,6380.0
8413,h,S,Western Metropolitan,3,12.6,3020.0,3.0,1.0,1.0,555.0,NaN,NaN,-37.79880,144.8220,3755.0


Echamos un vistazo a los datos de entrenamiento con el método `head()` a continuación.

Observe que los datos contienen tanto datos categóricos como columnas con valores perdidos.

¡Con una tubería, es fácil lidiar con ambos!

Construimos la tubería (pipeline) completa en tres pasos.

## Paso 1: definir los pasos de preprocesamiento

De manera similar a cómo una canalización agrupa los pasos de preprocesamiento y modelado, usamos la clase `ColumnTransformer` para agrupar diferentes pasos de preprocesamiento.

El siguiente código:

* imputa valores perdidos en datos **numéricos** e   
* imputa valores perdidos y aplica una codificación one-hot a datos **categóricos**.

In [11]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder

## Preprocesamiento de datos numéricos

In [12]:
numerical_transformer = SimpleImputer(strategy='constant')

In [ ]:
# Preprocessing for categorical data
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

## Bundle preprocessing for numerical and categorical data

## Procesamiento previo de paquetes para datos numéricos y categóricos

In [ ]:
# Bundle preprocessing for numerical and categorical data
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_cols),
        ('cat', categorical_transformer, categorical_cols)
    ])

## Paso 2: Definir el modelo

A continuación, definimos un modelo de bosque aleatorio con la clase familiar `RandomForestRegressor`.

In [ ]:
from sklearn.ensemble import RandomForestRegressor

model = RandomForestRegressor(n_estimators=100, random_state=0)

## Paso 3: crear y evaluar la canalización

Finalmente, usamos la clase Pipeline para definir una tubería que agrupa los pasos de preprocesamiento y modelado.

Hay algunas cosas importantes a tener en cuenta:

Con la canalización, preprocesamos los datos de entrenamiento y ajustamos el modelo en una sola línea de código.

(Por el contrario, sin una canalización, tenemos que realizar la imputación, la codificación one-hot y el entrenamiento del modelo en pasos separados.

¡Esto se vuelve especialmente complicado si tenemos que lidiar con variables tanto numéricas como categóricas!)

Con la canalización, proporcionamos las características sin procesar en `X_valid` al comando `predict()`, y la canalización preprocesa automáticamente las características antes de generar predicciones.

(Sin embargo, sin una canalización, debemos recordar preprocesar los datos de validación antes de hacer predicciones).

In [13]:
from sklearn.metrics import mean_absolute_error

In [ ]:
# Bundle preprocessing and modeling code in a pipeline
my_pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                              ('model', model)
                             ])

# Preprocessing of training data, fit model 
my_pipeline.fit(X_train, y_train)

# Preprocessing of validation data, get predictions
preds = my_pipeline.predict(X_valid)

# Evaluate the model
score = mean_absolute_error(y_valid, preds)
print('MAE:', score)

## Conclusión

Pipelines are valuable for cleaning up machine learning code and avoiding errors, and are especially useful for workflows with sophisticated data preprocessing.

## Your Turn

Utilice una canalización en el [siguiente ejercicio](https://www.kaggle.com/marcocanas/exercise-pipelines/edit) para utilizar técnicas avanzadas de procesamiento previo de datos y mejorar sus predicciones.